In [2]:
import pandas as pd

In [3]:
data_1 = pd.read_csv("input data_SD/1/1-000_00B4328B.txt.csv")

In [4]:
data_1

,PacketCounter,SampleTimeFine,Acc_X,Acc_Y,Acc_Z,FreeAcc_X,FreeAcc_Y,FreeAcc_Z,Gyr_X,Gyr_Y,...,OriInc_q3,Roll,Pitch,Yaw,Latitude,Longitude,Altitude,Vel_X,Vel_Y,Vel_Z
0,24441,,9.771520,0.096833,-2.177105,0.000000,-0.000000,0.198894,-0.116263,0.058676,...,-0.000039,177.499915,-77.410876,102.503736,,,0.0,,,
1,24442,,9.767511,0.191424,-2.084610,0.000000,0.000000,0.176633,-0.088354,0.048256,...,0.000004,174.783585,-77.889538,104.195258,,,0.0,,,
2,24443,,9.711458,0.152523,-2.162954,0.017940,-0.038539,0.137802,-0.059092,0.059591,...,-0.000007,176.169496,-77.670557,103.315050,,,0.0,,,
3,24444,,9.740847,0.240757,-2.076867,0.089121,0.059946,0.149436,-0.041482,0.040997,...,-0.000039,176.118247,-77.693695,103.343179,,,0.0,,,
4,24445,,9.870010,0.338038,-2.033783,0.170338,0.138607,0.267958,-0.096290,0.065170,...,-0.000067,176.016452,-77.730115,103.390713,,,0.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,26988,,9.756174,0.185038,-2.209723,-0.041533,-0.253946,0.189004,-0.089305,-0.003809,...,0.000022,171.849330,-78.487447,111.023700,,,0.0,,,
2548,26989,,9.711251,0.149098,-1.832761,-0.156135,0.090200,0.069474,-0.106766,0.002857,...,0.000022,171.803877,-78.486139,111.005009,,,0.0,,,
2549,26990,,9.684901,0.152882,-1.670896,-0.186018,0.240607,0.011776,-0.129166,-0.001431,...,0.000057,171.776742,-78.486341,110.956338,,,0.0,,,
2550,26991,,9.667065,0.124696,-1.669508,-0.213880,0.232246,-0.006808,-0.138361,0.001634,...,0.000035,171.717369,-78.485380,110.934584,,,0.0,,,
